In [2]:
%cd semantic_code_search/code_serach/modeling

/teamspace/studios/this_studio/semantic_code_search/code_serach/modeling


In [7]:
from omegaconf import OmegaConf
from model_manager import *

In [12]:
conf = OmegaConf.load("/teamspace/studios/this_studio/semantic_code_search/conf/modeling.yaml")

{'model_id': 'jinaai/jina-embeddings-v2-base-code', 'fine_tuning': {'full': {'layers': 2, 'train_embeddings': False}, 'peft': {'lora': {'config': {'task_type': 'SEQ_CLS', 'inference_mode': False, 'target_modules': ['q_proj', 'v_proj', 'k_proj'], 'r': 8, 'lora_alpha': 32, 'lora_dropout': 0.05}}, 'prefix': {'config': {'task_type': 'SEQ_CLS', 'num_virtual_tokens': 20, 'inference_mode': False, 'prefix_projection': True}}}}}